In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd

import urllib.request
import shutil

import re

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
# Get redlining and cdc data
# cdc data is stored locally because the PLACES API is restricted to 1000 rows
redline_data = gpd.read_file("https://dsl.richmond.edu/panorama/redlining/static/mappinginequality.gpkg")
cdc_data = gpd.read_file("data/PLACES__Local_Data_for_Better_Health__County_Data_2024_release_20250814.csv")

In [3]:
redline_data["area_id"].value_counts()

,count
area_id,
6513,1
244,1
193,1
206,1
203,1
...,...
192,1
243,1
195,1


In [4]:
redline_data

,area_id,city,state,city_survey,category,grade,label,residential,commercial,industrial,fill,geometry
0,244,Birmingham,AL,True,Best,A,A1,True,False,False,#76a865,"MULTIPOLYGON (((-86.75678 33.49754, -86.75653 ..."
1,193,Birmingham,AL,True,Best,A,A2,True,False,False,#76a865,"MULTIPOLYGON (((-86.75867 33.50933, -86.76134 ..."
2,206,Birmingham,AL,True,Best,A,A3,True,False,False,#76a865,"MULTIPOLYGON (((-86.75678 33.49754, -86.75692 ..."
3,203,Birmingham,AL,True,Still Desirable,B,B1,True,False,False,#7cb5bd,"MULTIPOLYGON (((-86.80111 33.48071, -86.80505 ..."
4,189,Birmingham,AL,True,Still Desirable,B,B10,True,False,False,#7cb5bd,"MULTIPOLYGON (((-86.74923 33.53333, -86.74971 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
10149,6518,Wheeling,WV,True,Hazardous,D,D4,True,False,False,#d9838d,"MULTIPOLYGON (((-80.71985 40.06376, -80.71943 ..."
10150,6520,Wheeling,WV,True,Hazardous,D,D5,True,False,False,#d9838d,"MULTIPOLYGON (((-80.72854 40.06729, -80.7287 4..."
10151,6516,Wheeling,WV,True,Hazardous,D,D6,True,False,False,#d9838d,"MULTIPOLYGON (((-80.72216 40.06134, -80.72247 ..."
10152,6512,Wheeling,WV,True,Hazardous,D,D7,True,False,False,#d9838d,"MULTIPOLYGON (((-80.6474 40.04886, -80.64842 4..."


In [5]:
cdc_data
# the geolocation is [longitude, latitude]

,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,...,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,TotalPop18plus,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text,Geolocation
0,2022,AL,Alabama,Clay,BRFSS,Health Outcomes,Current asthma among adults,%,Crude prevalence,11.0,...,9.6,12.4,14198,11235,01027,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,POINT (-85.8606604130173 33.2693085517833)
1,2022,AL,Alabama,Dale,BRFSS,Health Outcomes,Arthritis among adults,%,Crude prevalence,35.6,...,34.8,36.4,49544,38039,01045,HLTHOUT,ARTHRITIS,CrdPrv,Arthritis,POINT (-85.6109266826185 31.4316629601359)
2,2022,AL,Alabama,Jackson,BRFSS,Health Outcomes,Stroke among adults,%,Crude prevalence,5.1,...,4.6,5.6,52891,42087,01071,HLTHOUT,STROKE,CrdPrv,Stroke,POINT (-85.9995052603961 34.7796392771775)
3,2022,AL,Alabama,Lauderdale,BRFSS,Health Outcomes,Obesity among adults,%,Age-adjusted prevalence,36.5,...,29.6,43.9,95878,77472,01077,HLTHOUT,OBESITY,AgeAdjPrv,Obesity,POINT (-87.6540983850983 34.9014438477101)
4,2022,AL,Alabama,Lawrence,BRFSS,Disability,Any disability among adults,%,Crude prevalence,40.1,...,35.5,44.7,33214,26022,01079,DISABLT,DISABILITY,CrdPrv,Any Disability,POINT (-87.3108851040374 34.5216735395968)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240881,2021,WI,Wisconsin,Polk,BRFSS,Health Outcomes,High blood pressure among adults,%,Crude prevalence,34.3,...,29.9,38.8,45709,36755,55095,HLTHOUT,BPHIGH,CrdPrv,High Blood Pressure,POINT (-92.4412755945684 45.4615053795217)
240882,2022,WI,Wisconsin,Trempealeau,BRFSS,Health Outcomes,Depression among adults,%,Age-adjusted prevalence,24.5,...,20.9,28.2,30899,23116,55121,HLTHOUT,DEPRESSION,AgeAdjPrv,Depression,POINT (-91.3584214806691 44.3039450660913)
240883,2022,WI,Wisconsin,Door,BRFSS,Prevention,Visited dentist or dental clinic in the past y...,%,Age-adjusted prevalence,64.3,...,60.1,67.8,30526,25807,55029,PREVENT,DENTAL,AgeAdjPrv,Dental Visit,POINT (-87.3114193001272 44.9500144269812)
240884,2022,WI,Wisconsin,Marathon,BRFSS,Disability,Self-care disability among adults,%,Crude prevalence,3.2,...,2.9,3.5,137958,107333,55073,DISABLT,SELFCARE,CrdPrv,Self-care Disability,POINT (-89.7588560093353 44.8983004431375)


In [6]:
# These take a while; uncomment to see.
# redline_data.plot()

In [7]:
# convert cdc data into a proper geodataframe with geolocation points properly parsed
# i hate to have had to do this, given it was already written so close to the desired format...
cdc_data = gpd.GeoDataFrame(cdc_data, geometry = gpd.points_from_xy(cdc_data['Geolocation'].str.extract(r'POINT \(([-\d\.]+) ([-\d\.]+)\)')[0], cdc_data['Geolocation'].str.extract(r'POINT \(([-\d\.]+) ([-\d\.]+)\)')[1]))

In [8]:
# These take a while; uncomment to see.
# cdc_data.plot()

In [9]:
cdc_data = cdc_data.drop(["Geolocation"], axis=1)
cdc_data.head()

,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,...,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,TotalPop18plus,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text,geometry
0,2022,AL,Alabama,Clay,BRFSS,Health Outcomes,Current asthma among adults,%,Crude prevalence,11.0,...,9.6,12.4,14198,11235,01027,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,POINT (-85.86066 33.26931)
1,2022,AL,Alabama,Dale,BRFSS,Health Outcomes,Arthritis among adults,%,Crude prevalence,35.6,...,34.8,36.4,49544,38039,01045,HLTHOUT,ARTHRITIS,CrdPrv,Arthritis,POINT (-85.61093 31.43166)
2,2022,AL,Alabama,Jackson,BRFSS,Health Outcomes,Stroke among adults,%,Crude prevalence,5.1,...,4.6,5.6,52891,42087,01071,HLTHOUT,STROKE,CrdPrv,Stroke,POINT (-85.99951 34.77964)
3,2022,AL,Alabama,Lauderdale,BRFSS,Health Outcomes,Obesity among adults,%,Age-adjusted prevalence,36.5,...,29.6,43.9,95878,77472,01077,HLTHOUT,OBESITY,AgeAdjPrv,Obesity,POINT (-87.6541 34.90144)
4,2022,AL,Alabama,Lawrence,BRFSS,Disability,Any disability among adults,%,Crude prevalence,40.1,...,35.5,44.7,33214,26022,01079,DISABLT,DISABILITY,CrdPrv,Any Disability,POINT (-87.31089 34.52167)


In [10]:
# this data's crs is WGS84, so let's assign it
cdc_data.crs = "EPSG:4326"

In [11]:
# reproject redline data to CDC coordinates
redline_data = redline_data.to_crs(cdc_data.crs)

In [12]:
# now we reproject the cdc data again to add a buffer
# https://stackoverflow.com/questions/72073417/userwarning-geometry-is-in-a-geographic-crs-results-from-buffer-are-likely-i

cdc_data = cdc_data.to_crs(crs=3857)
cdc_data["geometry"] = cdc_data.buffer(1000.0) # The bigger the number, the less precise, but also more data we get, I think.
cdc_data = cdc_data.to_crs(redline_data.crs)

In [13]:
# These take a while; uncomment to see.
# cdc_data.plot()

In [14]:
# check whether each point in the CDC data falls into the redlining geometry
cdc_redline_intersects = gpd.overlay(cdc_data, redline_data, how='intersection')

In [15]:
# create an overlapping map to visualize the above intersections
# These take a while; uncomment to see.
# cdc_redline_intersects.plot()

In [16]:
# the remaining data should be just the parts of the CDC data that are included in the redlining data
cdc_redline_intersects.shape

(17030, 33)

In [17]:
# The geometry itself matters very little; the big gain from creating the GeoDataFrame above is figuring out locations overlap.
# list all column names
cdc_redline_intersects.columns

Index(['Year', 'StateAbbr', 'StateDesc', 'LocationName', 'DataSource',
       'Category', 'Measure', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value', 'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Low_Confidence_Limit', 'High_Confidence_Limit', 'TotalPopulation',
       'TotalPop18plus', 'LocationID', 'CategoryID', 'MeasureId',
       'DataValueTypeID', 'Short_Question_Text', 'area_id', 'city', 'state',
       'city_survey', 'category', 'grade', 'label', 'residential',
       'commercial', 'industrial', 'fill', 'geometry'],
      dtype='object')

In [18]:
# Year
cdc_redline_intersects["Year"].value_counts()

,count
Year,
2022,15238
2021,1792


In [19]:
# there's no reason to keep both StateAbbr and StateDesc, so we'll drop StateAbbr
cdc_redline_intersects = cdc_redline_intersects.drop(["StateAbbr"], axis=1)
# now we create dummies for state
cdc_redline_intersects = pd.get_dummies(cdc_redline_intersects, columns=["StateDesc"])

In [20]:
# location ID and LocationName both describe the county
# location ID is less readable, so let's drop it
cdc_redline_intersects = cdc_redline_intersects.drop(["LocationID"], axis=1)
# dummies for location name, as above
cdc_redline_intersects = pd.get_dummies(cdc_redline_intersects, columns=["LocationName"])

In [21]:
print(cdc_redline_intersects["DataSource"].value_counts())
# all the data is from the same source; this column should be dropped
cdc_redline_intersects = cdc_redline_intersects.drop(["DataSource"], axis=1)

DataSource
BRFSS    17030
Name: count, dtype: int64


In [22]:
# I'm going to drop the confidence limits columns, since it seems wrong to use them as predictive values.
cdc_redline_intersects = cdc_redline_intersects.drop(["Low_Confidence_Limit", "High_Confidence_Limit"], axis=1)

In [23]:
# TotalPopulation and TotalPop18Plus are fine as-is.
# 'CategoryID', 'MeasureId', 'DataValueTypeID' are redundant.
cdc_redline_intersects = cdc_redline_intersects.drop(["CategoryID", "MeasureId", "DataValueTypeID"], axis=1)

In [24]:
# short question text, redundant with measure
cdc_redline_intersects = cdc_redline_intersects.drop(["Short_Question_Text"], axis=1)

In [25]:
# Area ID is useful, but not here.
# cdc_redline_intersects = cdc_redline_intersects.drop(["area_id"], axis=1)

In [26]:
# Create dummies for city.
cdc_redline_intersects = pd.get_dummies(cdc_redline_intersects, columns=["city"])

In [27]:
# State is redundant with StateDesc.
cdc_redline_intersects = cdc_redline_intersects.drop(["state"], axis=1)

In [28]:
# Category is just a description of Grade and can be dropped.
cdc_redline_intersects = cdc_redline_intersects.drop(["category"], axis=1)

In [29]:
# Label is not useful.
cdc_redline_intersects = cdc_redline_intersects.drop(["label"], axis=1)

In [30]:
# Fill and geometry are no longer relevant and can also be dropped.
cdc_redline_intersects = cdc_redline_intersects.drop(["fill", "geometry"], axis=1)

In [31]:
# Without fill and geometry, we can convert the GeoDataFrame into a normal DataFrame.
cdc_redline_intersects = pd.DataFrame(cdc_redline_intersects)

In [32]:
cdc_redline_intersects["Data_Value_Type"].value_counts()

,count
Data_Value_Type,
Crude prevalence,8515
Age-adjusted prevalence,8515


In [33]:
# Unfortunately, we'll be shrinking our data quite a lot with the next few steps
# Drop all rows where the Data Value Type is Crude Prevalence, as we already have age-adjusted equivalents.
cdc_redline_intersects = cdc_redline_intersects[cdc_redline_intersects["Data_Value_Type"] != "Crude prevalence"]

In [34]:
# Now we drop all the remaining rows describing the data value (except the main one.)
# All values are expressed as percentages.
cdc_redline_intersects = cdc_redline_intersects.drop(["Data_Value_Type", "Data_Value_Unit", "Data_Value_Footnote_Symbol", "Data_Value_Footnote"], axis=1)

In [35]:
# And we'll also be dropping category; we only need measure.
cdc_redline_intersects = cdc_redline_intersects.drop(["Category"], axis=1)

In [36]:
# Drop area_id; it's somewhat redundant...
cdc_redline_intersects = cdc_redline_intersects.drop(["area_id"], axis=1)

In [37]:
cdc_redline_intersects["grade"].value_counts()

,count
grade,
C,2362
D,1952
B,1302
A,824
,610
E,33


In [38]:
# We'll drop rows with missing values now.
cdc_redline_intersects = cdc_redline_intersects.dropna()

In [39]:
# We'll be using mapping to turn the grade into numbers.
cdc_redline_intersects["grade"] = cdc_redline_intersects["grade"].map({"A": 4, "B": 3, "C": 2, "D": 1, "E": 0})

In [78]:
# We'll convert all "True" and "False" into booleans
cdc_redline_intersects = cdc_redline_intersects.replace({"True": 1, "False": 0})

In [79]:
cdc_redline_intersects["Measure"].value_counts()

,count
Measure,
Stroke among adults,189
Arthritis among adults,189
Frequent mental distress among adults,189
Current asthma among adults,189
Coronary heart disease among adults,189
Fair or poor self-rated health status among adults,189
No leisure-time physical activity among adults,189
Binge drinking among adults,189
Visits to doctor for routine checkup within the past year among adults,189


In [80]:
# As much as I dislike it, we have very few datapoints per measure.
# Still, let's choose one and make a VERY BAD model with it.
cdc_redline_asthma = cdc_redline_intersects[cdc_redline_intersects["Measure"] == "Current asthma among adults"]

In [81]:
cdc_redline_asthma = cdc_redline_asthma.drop(["Measure"], axis=1)

In [100]:
# Now we can finally train a baseline model.
X_train, X_test, y_train, y_test = train_test_split(cdc_redline_asthma.drop(["Data_Value"], axis=1), cdc_redline_asthma["Data_Value"], test_size=0.2)

In [101]:
baseline_asthma_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

In [102]:
baseline_asthma_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])

In [103]:
# Now let's get the accuracy.
baseline_y_score = baseline_asthma_pipe.score(X_test, y_test)
baseline_y_score

0.7681959612388529

In [115]:
# Now put it into a readable format
%load_ext google.colab.data_table
baseline_asthma_coefficients_dataframe = pd.DataFrame(abs(baseline_asthma_pipe.named_steps["model"].coef_), index=X_train.columns, columns=["coef"]).sort_values("coef", ascending=False)
baseline_asthma_coefficients_dataframe

,coef
LocationName_San Francisco,0.132018
city_San Francisco,0.132018
StateDesc_California,0.121389
LocationName_Bronx,0.112204
city_Bronx,0.112204
...,...
city_Lincoln,0.000000
city_Petersburg,0.000000
city_South McAlester,0.000000
city_Spokane,0.000000


In [116]:
# Grade is not high up there, but my methodology is *heavily* flawed.

In [119]:
# Next, we'll create the same model with only "grade" and "Data_Value".
cdc_redline_asthma_simple = cdc_redline_asthma[["grade", "Data_Value"]]

In [120]:
X_train, X_test, y_train, y_test = train_test_split(cdc_redline_asthma_simple.drop(["Data_Value"], axis=1), cdc_redline_asthma_simple["Data_Value"], test_size=0.2)

In [121]:
baseline_asthma_simple_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

In [122]:
baseline_asthma_simple_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])

In [123]:
baseline_asthma_simple_pipe.score(X_test, y_test)

-0.21359768785971545